In [ ]:
!pip install ultralytics
!pip install tensorflow
!pip install tensorflow-gpu
!pip install tensorflow-addons


In [1]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)


REPLICAS:  1


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Путь к вашей папке с данными
DATASET_PATH = 'datasets'

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]

# Загрузка данных из папок
train_dataset = image_dataset_from_directory(DATASET_PATH,
                                             validation_split=0.2,
                                             subset="training",
                                             seed=123,
                                             image_size=IMAGE_SIZE,
                                             batch_size=BATCH_SIZE)

validation_dataset = image_dataset_from_directory(DATASET_PATH,
                                                  validation_split=0.2,
                                                  subset="validation",
                                                  seed=123,
                                                  image_size=IMAGE_SIZE,
                                                  batch_size=BATCH_SIZE)

# Префетчинг данных для повышения производительности
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)


Found 5859 files belonging to 6 classes.
Using 4688 files for training.
Found 5859 files belonging to 6 classes.
Using 1171 files for validation.


In [5]:
from ultralytics import YOLO

# Загрузка предобученной модели YOLOv8
model = YOLO('yolov8n.pt')

# Настройка обучения
model.train(data='dataset.yaml', epochs=10, batch=BATCH_SIZE, device='tpu')

# Для использования TPU через TensorFlow
with strategy.scope():
    model.train(data='dataset.yaml', epochs=10, batch=BATCH_SIZE)


Ultralytics YOLOv8.2.32  Python-3.10.6 torch-2.3.1+cpu 


ValueError: Invalid CUDA 'device=tpu' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: tpu
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
# Оценка модели
results = model.val()

# Визуализация результатов
model.show_results()


In [7]:
import tensorflow_hub as hub
from tensorflow.keras import layers, models

# Загрузка модели EfficientDet
def build_model(num_classes):
    model = models.Sequential([
        hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1", trainable=True),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

NUM_CLASSES = 5  # количество классов (кухня, спальня, ванная и т.д.)

with strategy.scope():
    model = build_model(NUM_CLASSES)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

model.summary()


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x000002E9FF063280> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
EPOCHS = 10

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)


In [ ]:
# Оценка модели
loss, accuracy = model.evaluate(validation_dataset)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')


In [8]:
import tensorflow_hub as hub
from tensorflow.keras import layers, models, Input

# Загрузка модели EfficientDet
def build_model(num_classes, input_shape=IMAGE_SIZE + [3]):
    inputs = Input(shape=input_shape)
    efficientdet_model = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/d0/1", trainable=True)(inputs)
    outputs = layers.Dense(num_classes, activation='softmax')(efficientdet_model)
    model = models.Model(inputs, outputs)
    return model

NUM_CLASSES = 5  # количество классов (кухня, спальня, ванная и т.д.)

with strategy.scope():
    model = build_model(NUM_CLASSES)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

model.summary()


ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.


TypeError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None, 512, 512, 3), dtype=float32, sparse=None, name=keras_tensor>,) and kwargs: {} for signature: (input_tensor: TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name=None)).

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTe

In [9]:
from tensorflow.keras import layers, models, Input

# Определение модели
def build_model(num_classes, input_shape=IMAGE_SIZE + [3]):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

NUM_CLASSES = 5  # количество классов (кухня, спальня, ванная и т.д.)

with strategy.scope():
    model = build_model(NUM_CLASSES)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

model.summary()


C:\Users\stima\AppData\Local\Temp\ipykernel_40748\897891851.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 16, 16, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
EPOCHS = 2

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)


Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>

  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start

  File "C:\Users\stima\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\stima\AppData\Local\Prog

In [ ]:
# Оценка модели
loss, accuracy = model.evaluate(validation_dataset)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')


# Yolov5

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [7]:
# Yolov5
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Задайте путь к конфигурационному файлу
data_yaml = 'dataset_v2.yaml'

# Загрузите предобученную модель YOLOv5 (или YOLOv8)
model = YOLO('yolov5s.pt')  # или 'yolov8s.pt' для YOLOv8

# Параметры обучения
epochs = 50
batch = 16
img_size = 640

# Запуск обучения с указанием директории для сохранения модели
model.train(data=data_yaml, epochs=1, batch=batch, imgsz=img_size, project='runs/train', name='exp_custom')

# Загрузка лучшей модели после обучения
best_model_path = 'runs/train/exp_custom/weights/best.pt'
model = YOLO(best_model_path)

# Валидация модели
results = model.val(data=data_yaml, imgsz=img_size)

# Доступ к метрикам
precision = results.metrics['precision']
recall = results.metrics['recall']
mAP_0_5 = results.metrics['map_50']
mAP_0_5_0_95 = results.metrics['map']

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"mAP@0.5: {mAP_0_5:.4f}")
print(f"mAP@0.5:0.95: {mAP_0_5_0_95:.4f}")

# Визуализация результатов
results.plot()
plt.show()



PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.
New https://pypi.org/project/ultralytics/8.2.49 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.32  Python-3.10.6 torch-2.3.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov5s.pt, data=dataset_v2.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=exp_custom5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json

train: Scanning C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\train\backyard.cache... 0 images, 5810 backgrounds, 0 corrupt: 100%|██████████| 5810/5810 [00:00<?, ?it/s]Scanning C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\train\backyard.cache... 0 images, 5810 backgrounds, 0 corrupt: 100%|██████████| 5810/5810 [00:00<?, ?it/s]


WARNING  No labels found in C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\train\backyard.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: Scanning C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\val\backyard.cache... 0 images, 35 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]Scanning C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\val\backyard.cache... 0 images, 35 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]


WARNING  No labels found in C:\Users\stima\PycharmProjects\ML\Room-type-image-classification\dataset_yolov_v2\val\backyard.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs\train\exp_custom5\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)


2024/07/05 15:05:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/07/05 15:05:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/07/05 15:05:50 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of keras. If you encounter errors during autologging, try upgrading / downgrading keras to a supported version, or try upgrading MLflow.
2024/07/05 15:05:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


MLflow: logging run_id(61e0cb5eafaf4aac9620e2aa054de262) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
MLflow: WARNING  Failed to initialize: Changing param values is not allowed. Param with key='epochs' was already logged with value='50' for run ID='61e0cb5eafaf4aac9620e2aa054de262'. Attempted logging new value '1'.
MLflow: WARNING  Not tracking this run
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\train\exp_custom5
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/364 [01:42<?, ?it/s]


KeyboardInterrupt: 